<a href="https://colab.research.google.com/github/Bio2Byte/public_notebooks/blob/main/Bio2Byte_Pymol_scripting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up

In [ ]:
#@title 1. Preparing the online environment
#@markdown This cell will set up this online environment to run PyMol scripts.
#@markdown ## Notes
#@markdown It may be required to restart the Kernel, do not worry, it is normal.

%%capture

!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title 2. Installing PyMol library
#@markdown This cell will install the PyMol library and its dependencies. This 
#@markdown step may take a long while to finish.

%%capture
!conda install schrodinger::pymol

In [ ]:
#@title 3. Creating the results directory
#@markdown This cell will create a directory to store all the files generated by
#@markdown PyMol
%%capture

import os

if not os.path.exists("/content/results"):
    os.mkdir("/content/results")

# PyMol scripting

In [ ]:
#@title 4. PDB input
#@markdown Type here the name of the PDB of interest.
import os 

pdb_name = '6s7o' #@param {type:"string"}

image_filename        = "/content/results/{0}.png".format(pdb_name)
animation_filename    = "/content/results/{0}.gif".format(pdb_name)
animation_temp_prefix = "/tmp/{0}/".format(pdb_name)

if not os.path.exists(animation_temp_prefix):
    os.mkdir(animation_temp_prefix)

In [ ]:
#@title 5.a Creating a static image of the 3D structure 
%%capture

from pymol import cmd
from IPython import display

# 1. Load a PDB file of your choice
cmd.fetch(pdb_name)
cmd.show_as("cartoon")
cmd.center()

# 2. Export the image as PNG
cmd.exporting.save(filename=image_filename, format="png")

In [ ]:
#@title 5.b Rendering the static image of the 3D structure 

display.Image(image_filename, format='png', width='400')

In [ ]:
#@title 6.a Rotating 360 degrees the 3D structure 
%%capture

import os
import imageio

# Number of degrees to rotate each frame
step = 10

# Vector to hold images
images = []

for a in range(0, 360, step):
  cmd.rotate("y", float(step)) # Rotate around Y-axis
  # cmd.ray(256,256) # Raytrace 256x256 image
  
  filename = os.path.join(animation_temp_prefix, "{0}.png".format(a))
  cmd.exporting.save(filename, format="png")

  images.append(imageio.imread(filename))

# Create gif animation from images
imageio.mimsave(animation_filename, images)

In [ ]:
#@title 6.b Rendering the 360-degree rotated 3D structure 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython import display

display.Image(open(animation_filename,'rb').read())

In [ ]:
#@title 7. Download the results
#@markdown This step will export all the results stored inside `/content/results` directory. 

from google.colab import files

!zip -r /content/pymol-results.zip /content/results

files.download("/content/pymol-results.zip")